In [15]:
!pip install geopy


Looking in indexes: http://jfrog.fkinternal.com/artifactory/api/pypi/python_virtual/simple
     |################################| 118 kB 3.4 MB/s            


In [17]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from torch import nn
import os
import json
import time
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from h3 import h3
from geopy.distance import geodesic as vincenty
from tqdm import tqdm

In [8]:
print("This is evaluation task")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

This is evaluation task
cuda


In [10]:
address_data_path = '/home/uddeshya.singh/DATA/PUNJAB_test/PUNJAB_with_grid_ids_test.csv' 
fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/level-two_models/Fine-tuned-save'
model_dir =  "/home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)


In [24]:
class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_labels_res8, num_labels_res10, num_labels_res9):
        super(MultiTaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier_res8 = nn.Linear(self.hidden_size, num_labels_res8)
        self.classifier_res10 = nn.Linear(self.hidden_size, num_labels_res10)
        self.classifier_res9 = nn.Linear(self.hidden_size, num_labels_res9)
    
    def forward(self, input_ids, attention_mask=None, labels_res8=None, labels_res10=None, labels_res9=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        logits_res8 = self.classifier_res8(pooled_output)
        logits_res10 = self.classifier_res10(pooled_output)
        logits_res9 = self.classifier_res9(pooled_output)

        return {"logits_res10": logits_res10, "logits_res8": logits_res8, "logits_res9": logits_res9}


In [18]:
with open(os.path.join(fine_tuned_lm_trained_save_dir, 'config.json'), 'r') as f:
    config = json.load(f)

num_labels_res8 = config['num_labels_res8']
num_labels_res10 = config['num_labels_res10']
num_labels_res9 = config['num_labels_res9']

In [19]:
num_labels_res8

18075

In [20]:
num_labels_res10

161418

In [21]:
num_labels_res9

52558

In [27]:
model = MultiTaskModel(model_dir, num_labels_res8, num_labels_res10, num_labels_res9)


Some weights of the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN thi

In [28]:
state_dict = torch.load(os.path.join(fine_tuned_lm_trained_save_dir, 'MM_model.pt'))
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith('module.') else k  # remove 'module.' prefix
    new_state_dict[name] = v

In [29]:
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [30]:
model.to(device)
model.eval()

MultiTaskModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,

In [31]:
print('Loading dataset...')
test_df = pd.read_csv(address_data_path)
print(list(test_df), len(test_df))

Loading dataset...
['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id'] 184099


In [39]:
test_df.head()

addr_hash  \
0  c5cf1f7511c2ae3e385bf9f8f846317c   
1  b5a0c40241038bd9d410a91df433926b   
2  3104b020a7131e005a5c68a87ae18dd4   
3  04aab31bb493bc0fca2619c29fdbc894   
4  70126c55ada9a880de480a923e237cf7   

                                     cleaned_address  lookup_lat  lookup_lng  \
0   0 sukhram bagh near model town railon road ropar   30.952042   76.523861   
1   01881 502433 c o mr sangat singh h no 4684 op...   30.973804   76.517214   
2                         02 shampura krishna colony   30.958224   76.507985   
3   1 canal rest house roper punjab nearby dc office   30.981542   76.517667   
4                              1 m head works colony   30.984002   76.521956   

           grid_id  
0  8a3d14716cb7fff  
1  8a3d14449aa7fff  
2  8a3d144494b7fff  
3  8a3d147324cffff  
4  8a3d14732637fff

In [33]:
def load_id2label(dir_local):
    save_id2label = "/home/uddeshya.singh/level-two_models/Fine-tuned-save/id2label.json"
    with open(save_id2label, 'r') as f:
        id2label = json.load(f)
    return id2label

def load_label2id(dir_local):
    save_label2id = "/home/uddeshya.singh/level-two_models/Fine-tuned-save/label2id.json"
    with open(save_label2id, 'r') as f:
        label2id = json.load(f)
    return label2id

id2label = load_id2label(fine_tuned_lm_trained_save_dir)
label2id = load_label2id(fine_tuned_lm_trained_save_dir)
print(len(id2label), len(label2id))

161420 161420


In [34]:
def tokenize(batch):
    return tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)

In [37]:
# Prepare the data
cleaned_address_list_test = test_df.cleaned_address.tolist()
fault_line_list = [ind for ind, a in enumerate(cleaned_address_list_test) if a is None or type(a) != str or pd.isnull(a)]
new_cleaned_address_list_test = [a for ind, a in enumerate(cleaned_address_list_test) if ind not in fault_line_list]

In [41]:
new_cleaned_address_list_test[:5]

[' 0 sukhram bagh near model town railon road ropar',
 ' 01881 502433 c o mr sangat singh h no 4684 opp krishna mandir haveli kalan just opposite fast way cabel office behind government girls school',
 ' 02 shampura krishna colony',
 ' 1 canal rest house roper punjab nearby dc office',
 ' 1 m head works colony']

In [48]:
def predict(batch):
    predictions = []
    for i in tqdm(range(0, len(batch), 128)):  # Using a batch size of 128 for prediction
        inputs = tokenizer(batch[i:i+128], return_tensors="pt", padding=True, truncation=True, max_length=100).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits_res10 = outputs['logits_res9']
        batch_predictions = torch.argmax(logits_res10, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
    return predictions

In [43]:
print('Starting predictions...')
start_time = time.strftime('%X %x %Z')
initial_predictions = predict(new_cleaned_address_list_test)
print('Completed predictions ', time.strftime('%X %x %Z'))

Starting predictions...


100%|██████████| 1439/1439 [02:49<00:00,  8.48it/s]

Completed predictions  18:54:14 08/01/24 IST


In [44]:
def process_labels(predictions):
    pred_grid_list = []
    for p in predictions:
        grid_id = id2label[str(p)]
        pred_grid_list.append(grid_id)
    return pred_grid_list

pred_grid_list = process_labels(initial_predictions)
print(pred_grid_list[0:5])

['8a3d14716cb7fff', '8a3d1444916ffff', '8a3d144494a7fff', '8a3d14449617fff', '8a3d14449617fff']


In [51]:

pred_grid_list = process_labels(initial_predictions)
print(pred_grid_list[0:5])

TypeError: process_labels() missing 1 required positional argument: 'id2label'

In [45]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(vincenty((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred ", e)
        return None

def get_drift_for_row(row_local):
    pred_coor = get_centroid_of_grid(row_local['top_grid'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue
        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1
    
    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]
    
    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))
    
    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

# Apply predictions and calculate accuracy
test_df['top_grid'] = pred_grid_list
test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df.pred_drift.tolist())
print(test_drift_acc_dict)

{'num': {100: 112706, 200: 140214, 500: 163864, 1000: 171476, 2000: 174450, 'rest': 9649, 'None': 0}, 'per': {100: 0.6122032167475109, 200: 0.7616228225031098, 500: 0.8900863122559058, 1000: 0.9314336308182011, 2000: 0.9475879825528656, 'rest': 0.052412017447134425, 'None': 0.0}}


In [46]:
save_dir = "/home/uddeshya.singh/level-two_models/Fine-tuned-save/"
def load_label_mappings(save_dir, resolution):
    save_id2label = os.path.join(save_dir, f'id2label_res{resolution}.json')
    with open(save_id2label, 'r') as f:
        id2label = json.load(f)
    
    save_label2id = os.path.join(save_dir, f'label2id_res{resolution}.json')
    with open(save_label2id, 'r') as f:
        label2id = json.load(f)
    
    return id2label, label2id


id2label_res8, label2id_res8 = load_label_mappings(fine_tuned_lm_trained_save_dir, 8)
id2label_res9, label2id_res9 = load_label_mappings(fine_tuned_lm_trained_save_dir, 9)

In [50]:
# Function to process predictions for a given resolution
def process_labels(predictions, id2label):
    pred_grid_list = []
    for p in predictions:
        grid_id = id2label[str(p)]
        pred_grid_list.append(grid_id)
    return pred_grid_list

# Evaluate function with tqdm for progress tracking
def predict(batch, head='res10'):
    predictions = []
    for i in tqdm(range(0, len(batch), 128)):  # Using a batch size of 128 for prediction
        inputs = tokenizer(batch[i:i+128], return_tensors="pt", padding=True, truncation=True, max_length=100).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs[f'logits_{head}']
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
    return predictions

# Predict and evaluate for resolutions 8, 9, and 10
for resolution in [9]:
    print(f'Starting predictions for resolution {resolution}...')
    head = f'res{resolution}'
    id2label = globals()[f'id2label_{head}']
    
    initial_predictions = predict(new_cleaned_address_list_test, head=head)
    pred_grid_list = process_labels(initial_predictions, id2label)
    
    # Apply predictions and calculate accuracy
    test_df[f'top_grid_{head}'] = pred_grid_list
    test_df[f'pred_drift_{head}'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
    test_drift_acc_dict = get_accuracy_for_df(test_df[f'pred_drift_{head}'].tolist())
    print(f'Accuracy for resolution {resolution}:')
    print(test_drift_acc_dict)


Starting predictions for resolution 9...


100%|██████████| 1439/1439 [04:40<00:00,  5.13it/s]


Accuracy for resolution 9:
{'num': {100: 112706, 200: 140214, 500: 163864, 1000: 171476, 2000: 174450, 'rest': 9649, 'None': 0}, 'per': {100: 0.6122032167475109, 200: 0.7616228225031098, 500: 0.8900863122559058, 1000: 0.9314336308182011, 2000: 0.9475879825528656, 'rest': 0.052412017447134425, 'None': 0.0}}
